This notebook contains scripts for calculating kinematic data.
Input files must be `.csv` files generated by `rtklib-pos-to-csv.ipynb`.

In [21]:
import csv
import datetime as dt
import math
from geopy.distance import geodesic

In [22]:
TIMESTAMP_FORMAT = "%Y/%m/%d %H:%M:%S.%f"

INPUT_CSV_HEADER = ["timestamp", "latitude", "longitude", "altitude", "fix-type", "satellites", "sdn",  "sde",  "sdu", "sdne", "sdeu", "sdun", "age", "ratio"]
CALC_CSV_HEADER = [
    "timestamp",
    "fix-type",
    "latitude",
    "longitude",
    "ground-speed-raw", # m/s
    "ground-speed-processed", # m/s, TODO: should be smoothed
    "altitude", # meters
    "horizontal_accuracy",
    "vertical_accuracy",
    "speed_accuracy"
 ]

INPUT_FILEPATH = "/home/delaynie/Documents/rtklib-workspace/cmubuggy-datalogger/data/position-csv/"
INPUT_FILENAMES = ["output1.csv", "output2.csv", "output3.csv"]
OUTPUT_FILEPATH = "/home/delaynie/Documents/rtklib-workspace/cmubuggy-datalogger/data/calc-csv/"

In [23]:
MIN_SATELLITES = 2

In [24]:
def calc(inputPath, outputPath):
    # Create headeres in output path path
    with open(outputPath, 'w') as file:
        writer = csv.DictWriter(file, fieldnames=CALC_CSV_HEADER)
        writer.writeheader()
        file.close()

    lastLat = None
    lastLon = None
    lastVel = None
    lastTs = None

    with open(inputPath, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            if int(row["satellites"]) < MIN_SATELLITES: continue

            timestamp = dt.datetime.strptime(row["timestamp"], TIMESTAMP_FORMAT)
            lat = float(row["latitude"])
            lon = float(row["longitude"])
            alt = float(row["altitude"])

            sdn = float(row["sdn"])
            sde = float(row["sde"])
            sdu = float(row["sdu"])

            horizAcc = round(math.sqrt(sdn**2 + sde**2), 4) # TODO this is not statistically accurate
            vertAcc = round(sdu, 4) # TODO this may or may not need some math done on it

            # TODO fit some kind of curve to compute a smooth measurement of speed
            if lastLat and lastLon:
                diff = geodesic((lastLat, lastLon), (lat, lon)).meters # distance in meters
                timediff = (timestamp - lastTs).microseconds / 1000000 # seconds
                groundSpeedRaw = round((diff/timediff), 4) # m/s

                if lastVel:
                    acceleration = (groundSpeedRaw - lastVel) / (timediff) # m/s^2
                else:
                    acceleration = 0

                lastVel = groundSpeedRaw

                # if acceleration > 100 or groundSpeedRaw > 45:
                #     continue

                speedAcc = 0 # TODO

                with open(outputPath, 'a') as file:
                    csv.DictWriter(file, fieldnames=CALC_CSV_HEADER).writerow({
                        "timestamp": timestamp.strftime(TIMESTAMP_FORMAT),
                        "fix-type": row["fix-type"],
                        "latitude": lat,
                        "longitude": lon,
                        "ground-speed-raw": groundSpeedRaw, # m/s
                        "ground-speed-processed": groundSpeedRaw, # m/s, TODO: should be smoothed
                        "altitude": alt, # meters
                        "horizontal_accuracy": horizAcc,
                        "vertical_accuracy": vertAcc,
                        "speed_accuracy": speedAcc
                    })

            lastLat = lat
            lastLon = lon
            lastTs = timestamp



In [25]:
for name in INPUT_FILENAMES:
    calc(INPUT_FILEPATH+name, OUTPUT_FILEPATH+name)

ValueError: unconverted data remains: 936